In [6]:
## Import Python Packages and scripts
import sys, os, subprocess
import gen3

from gen3.auth import Gen3Auth # authentication SDK class
from gen3.query import Gen3Query # query SDK class

In [7]:
credentials = rf"C:\Users\Wahaj Sajid\Desktop\Research\credentials.json"
api = "https://data.midrc.org"

In [8]:
auth = Gen3Auth(api, refresh_file=credentials) # authentication class
query = Gen3Query(auth) # query class

In [ ]:
## "case" query parameters
## In this example, we're going to filter our patient cohort by asking for:
# female Asian patients in an age range that tested positive for COVID-19.

# demographic attributes / filters
race = "Asian"
sex = "Male"
min_age = 75
max_age = 100

# clinical attributes / filters
covid19_positive = "Yes"

# fields to return.  
fields = ["submitter_id", # "submitter_id" here is the case/patient's unique identifier in the database
    "project_id" # this is the "project" that the patient belongs to. by default, queries run across all projects
]

In [10]:
# Set a limit
limit = 500  # Download only 100 cases
## Run the query using the guppy graphQL service
data = query.raw_data_download(
                    data_type="case",
                    fields=fields,
                    filter_object={
                        "AND": [
                            {"=": {"race": race}},
                            {"=": {"sex": sex}},
                            {">=": {"age_at_index": min_age}},
                            {"<=": {"age_at_index": max_age}},
                            {"=": {"covid19_positive": covid19_positive}}
                        ]
                    },
                    sort_fields=[{"submitter_id": "asc"}]
                )

if len(data) > 0 and "submitter_id" in data[0]:
    data = data[:limit]
    case_ids = [i['submitter_id'] for i in data] ## make a list of the case (patient) IDs returned
    print("Query returned {} case IDs.".format(len(data)))
    print("Data is a list with rows like this:\n\t {}".format(data[0:1]))
else:
    print("Your query returned no data! Please, check that query parameters are valid.")

[2025-08-15 20:01:40,244][WARNING] failed to write token cache file: C:\Users\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81
[2025-08-15 20:01:40,244][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81.tmp_eraseme_23896_1755270100' -> 'C:\\Users\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81'
[2025-08-15 20:01:40,244][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x0000022CA3767BD0>, C:\Users\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81, eyJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV9rZXkiLCJ0eXAiOiJKV1QifQ.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2RhdGEubWlkcmMub3JnL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9kYXRhLm1pZHJjLm9yZy91c2VyIiwib3BlbmlkIiwiYWRtaW4iLCJnb29nbGVfY3Jl

In [11]:
## "data_file" query parameters
## In this example, we're asking for files from CT imaging studies of the chest

# imaging_study attributes / filters
source_node = "ct_series_file" # this will limit the files returned to those that are CT series
loinc_system = "Chest" # this is the LOINC-harmonized "body part examined" in the imaging study

# fields to return.  
fields = [
    "project_id", # this is the "project" that the file belongs to. by default, queries run across all projects
    "case_ids", # this is the "submitter_id" of the patient the file is associated with (the patient ID)
    "object_id", # this is the unique identifier (GUID) for a file in MIDRC which can be used to access/download the file
    "source_node", # this is the name of the node in the MIDRC data model under which the file is stored
    "file_name",
    "file_size"
]

In [12]:
# note that the field "data_type" here has changed from "case" (example above) to "data_file". This is the name of the Elasticsearch index
data = query.raw_data_download(
                    data_type="data_file",
                    fields=fields,
                    filter_object={
                        "AND": [
                            {"=": {"source_node": source_node}},
                            {"=": {"loinc_system": loinc_system}},
                            {"IN": {"case_ids": case_ids}},
                        ]
                    },
                    sort_fields=[{"submitter_id": "asc"}]
                )

if len(data) > 0 and "object_id" in data[0]:
    object_ids = [i['object_id'] for i in data] ## make a list of the file object_ids returned by our query
    print("Query returned {} data files with {} object_ids.".format(len(data),len(object_ids)))
    print("Data is a list with rows like this:\n\t {}".format(data[0:1]))
else:
    print("Your query returned no data! Please, check that query parameters are valid.")

Query returned 1266 data files with 1266 object_ids.
Data is a list with rows like this:
	 [{'case_ids': ['419639-000650'], 'project_id': 'Open-R1', 'file_name': '419639-000650/1.2.826.0.1.3680043.10.474.419639.162673109194151486050598414165/1.2.826.0.1.3680043.10.474.419639.108813202520505568217849264071.zip', 'source_node': 'ct_series_file', 'object_id': 'dg.MD1R/a9934f7b-e91b-445a-8eb0-a213283bcdf7', 'file_size': 2285542}]


In [13]:
source_nodes = ["ct_series_file", "annotation_file","dicom_annotation_file"]

In [14]:
# note that the field "data_type" here has changed from "case" (example above) to "data_file". This is the name of the Elasticsearch index
data = query.raw_data_download(
                    data_type="data_file",
                    fields=fields,
                    filter_object={
                        "AND": [
                            {"in": {"source_node": source_nodes}},
                            {"=": {"loinc_system": loinc_system}},
                            {"IN": {"case_ids": case_ids}},
                        ]
                    },
                    sort_fields=[{"submitter_id": "asc"}]
                )

if len(data) > 0:
    object_ids = [i['object_id'] for i in data if 'object_id' in i] ## make a list of the file object_ids returned by our query
    print("Query returned {} data files with {} object_ids.".format(len(data),len(object_ids)))
    print("Data is a list with rows like this:\n\t {}".format(data[0:1]))
else:
    print("Your query returned no data! Please, check that query parameters are valid.")

Query returned 2156 data files with 2156 object_ids.
Data is a list with rows like this:
	 [{'case_ids': ['419639-000550'], 'project_id': 'Open-R1', 'file_name': '419639-000550/1.2.826.0.1.3680043.10.474.419639.101383804482187466278196291808/1.2.826.0.1.3680043.10.474.419639.528328784716155109616542416331.zip', 'source_node': 'ct_series_file', 'object_id': 'dg.MD1R/8e4d042d-0335-45da-a061-ef470d74e047', 'file_size': 56885432}]


In [ ]:
output_dir = rf"C:\Users\Wahaj Sajid\Desktop\Asian men COVID 19 positve (Age  75 to 100) download data"

In [ ]:
## Simple loop to download all files and keep track of success and failures
object_id = object_ids[0]
success,failure,other=[],[],[]
count,total = 0,len(object_ids)
for object_id in object_ids:
    count+=1
    cmd = f'gen3 --auth "{credentials}" --endpoint data.midrc.org drs-pull object {object_id} --output-dir "{output_dir}"'
    stout = subprocess.run(cmd, shell=True, capture_output=True)
    print("Progress ({}/{}): {}".format(count,total,stout.stdout))
    if "failed" in str(stout.stdout):
        failure.append(object_id)
    elif "successfully" in str(stout.stdout):
        success.append(object_id)
    else:
        other.append(object_id)          

Progress (1/2156): b'[2025-08-15 20:02:20,304][WARNING] failed to write token cache file: C:\\Users\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81\r\n[2025-08-15 20:02:20,304][WARNING] [WinError 183] Cannot create a file when that file already exists: \'C:\\\\Users\\\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81.tmp_eraseme_59301_1755270140\' -> \'C:\\\\Users\\\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81\'\r\n[2025-08-15 20:02:20,304][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x00000125FF4AD990>, C:\\Users\\Wahaj Sajid/.cache/gen3/token_cache_750b4eb9e1145ccf60ffdf0af86340697e7d394de4e2a420bc189e9c0e3cda81, eyJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV9rZXkiLCJ0eXAiOiJKV1QifQ.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2RhdGEubWlkcmMub3JnL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9kYXRhLm1pZHJjLm9yZy91c